In [1]:
import numpy as np
import time
import re

In [2]:
import datetime

In [3]:
from html.parser import HTMLParser  
from urllib.request import urlopen  
from urllib import parse

# We are going to create a class called LinkParser that inherits some
# methods from HTMLParser which is why it is passed into the definition
class LinkParser(HTMLParser):

    # This is a function that HTMLParser normally has
    # but we are adding some functionality to it
    def handle_starttag(self, tag, attrs):
        # We are looking for the begining of a link. Links normally look
        # like <a href="www.someurl.com"></a>
        if tag == 'a':
            for (key, value) in attrs:
                if key == 'href':
                    # We are grabbing the new URL. We are also adding the
                    # base URL to it. For example:
                    # www.netinstructions.com is the base and
                    # somepage.html is the new URL (a relative URL)
                    #
                    # We combine a relative URL with the base URL to create
                    # an absolute URL like:
                    # www.netinstructions.com/somepage.html
                    newUrl = parse.urljoin(self.baseUrl, value)
                    # And add it to our colection of links:
                    self.links = self.links + [newUrl]

    # This is a new function that we are creating to get links
    # that our spider() function will call
    def getLinks(self, url):
        self.links = []
        # Remember the base URL which will be important when creating
        # absolute URLs
        self.baseUrl = url
        # Use the urlopen function from the standard Python 3 library
        response = urlopen(url)
        # Make sure that we are looking at HTML and not other things that
        # are floating around on the internet (such as
        # JavaScript files, CSS, or .PDFs for example)
        if 'text/html' in response.getheader('Content-Type'):
            htmlBytes = response.read()
            # Note that feed() handles Strings well, but not bytes
            # (A change from Python 2.x to Python 3.x)
            htmlString = htmlBytes.decode("utf-8")
            self.feed(htmlString)
            return htmlString, self.links
        else:
            return "",[]

# And finally here is our spider. It takes in an URL, a word to find,
# and the number of pages to search through before giving up
def spider(url, word, maxLvl):  
    pagesToVisit = list(url)
    pagesVisited = []
    numberVisited = 0
    numberLvl = 0
    lvl = 0
    foundWord = False
    # The main loop. Create a LinkParser and get all the links on the page.
    # Also search the page for the word or string
    # In our getLinks function we return the web page
    # (this is useful for searching for the word)
    # and we return a set of links from that web page
    # (this is useful for where to go next)
    while numberLvl < maxLvl and pagesToVisit != [] and not foundWord:
        numberVisited = numberVisited +1
#         print('Number Visited:', numberVisited, len(pagesToVisit))
        # Start from the beginning of our collection of pages to visit:
        if numberVisited == lvl+1:
            numberLvl += 1
        url = pagesToVisit[0]
        pagesToVisit = pagesToVisit[1:]

        fl = 0
#         for reg in check:
#             if re.match(reg, url) is not None:
#                 fl = 1
        for reg in check:
            if reg in url:
                fl = 1
                
        if fl == 0:
            continue
            
        if url in pagesVisited:
            continue
            
        try:
            print(numberVisited, "Visiting:", url)
            parser = LinkParser()
            data, links = parser.getLinks(url)
            
            pagesVisited.append(url)
            
            if 'document' in url:
                dt = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
                with open('files/file_' + dt + str(numberVisited), 'w') as f:
                    f.writelines(url)
                    f.writelines('')
                    f.write(data)
                continue
                
            pagesToVisit = pagesToVisit + links
            if numberVisited == lvl+1:
                print('NEW LVL BORD')
                lvl = len(pagesToVisit)
                print(lvl)
                print()
                
        except:
            print(" **Failed!**")
    if foundWord:
        print("The word", word, "was found at", url)
    else:
        print("Word never found")
    return (pagesToVisit, pagesVisited)

In [4]:
# check = ['https?://(www\\.)?([a-z\\-_]+\\.)?ieeexplore\\.ieee\\.org/xpl/tocresult.jsp.*$', 
#          'https?://(www\\.)?([a-z\\-_]+\\.)?ieeexplore\\.ieee\\.org/document.*$', 
#          'https?://(www\\.)?([a-z\\-_]+\\.)?ieeexplore\\.ieee\\.org/xpl/RecentIssue.*$']

check = ['tocresult.jsp', 
         'document']

In [ ]:
journals = ['https://ieeexplore.ieee.org/xpl/RecentIssue.jsp?punumber=83']
for journal in journals:
    parser = LinkParser()
    data, links = parser.getLinks(journal)
    print(journal, len(links))
    
    _time = time.time()
    pagesToVisit, pagesVisited = spider(links, '', 2)
    print(time.time() - _time)

https://ieeexplore.ieee.org/xpl/RecentIssue.jsp?punumber=83 960
43 Visiting: https://ieeexplore.ieee.org/xpl/tocresult.jsp?isnumber=4358840
78 Visiting: https://ieeexplore.ieee.org/xpl/tocresult.jsp?isnumber=6106034&punumber=83
79 Visiting: https://ieeexplore.ieee.org/xpl/tocresult.jsp?isnumber=6129825&punumber=83
80 Visiting: https://ieeexplore.ieee.org/xpl/tocresult.jsp?isnumber=6151934&punumber=83
81 Visiting: https://ieeexplore.ieee.org/xpl/tocresult.jsp?isnumber=6170603&punumber=83
82 Visiting: https://ieeexplore.ieee.org/xpl/tocresult.jsp?isnumber=6183042&punumber=83
83 Visiting: https://ieeexplore.ieee.org/xpl/tocresult.jsp?isnumber=6198806&punumber=83
84 Visiting: https://ieeexplore.ieee.org/xpl/tocresult.jsp?isnumber=6217330&punumber=83
85 Visiting: https://ieeexplore.ieee.org/xpl/tocresult.jsp?isnumber=6238420&punumber=83
86 Visiting: https://ieeexplore.ieee.org/xpl/tocresult.jsp?isnumber=6269121&punumber=83
87 Visiting: https://ieeexplore.ieee.org/xpl/tocresult.jsp?isnumber=

170 Visiting: https://ieeexplore.ieee.org/xpl/tocresult.jsp?isnumber=4286981&punumber=83
171 Visiting: https://ieeexplore.ieee.org/xpl/tocresult.jsp?isnumber=4303133&punumber=83
172 Visiting: https://ieeexplore.ieee.org/xpl/tocresult.jsp?isnumber=4337753&punumber=83
173 Visiting: https://ieeexplore.ieee.org/xpl/tocresult.jsp?isnumber=4376239&punumber=83
174 Visiting: https://ieeexplore.ieee.org/xpl/tocresult.jsp?isnumber=10299&punumber=83
175 Visiting: https://ieeexplore.ieee.org/xpl/tocresult.jsp?isnumber=10394&punumber=83
176 Visiting: https://ieeexplore.ieee.org/xpl/tocresult.jsp?isnumber=10547&punumber=83
177 Visiting: https://ieeexplore.ieee.org/xpl/tocresult.jsp?isnumber=10548&punumber=83
178 Visiting: https://ieeexplore.ieee.org/xpl/tocresult.jsp?isnumber=10607&punumber=83
179 Visiting: https://ieeexplore.ieee.org/xpl/tocresult.jsp?isnumber=10909&punumber=83
180 Visiting: https://ieeexplore.ieee.org/xpl/tocresult.jsp?isnumber=10981&punumber=83
181 Visiting: https://ieeexplore.ie

265 Visiting: https://ieeexplore.ieee.org/xpl/tocresult.jsp?isnumber=21845&punumber=83
266 Visiting: https://ieeexplore.ieee.org/xpl/tocresult.jsp?isnumber=21967&punumber=83
267 Visiting: https://ieeexplore.ieee.org/xpl/tocresult.jsp?isnumber=22043&punumber=83
268 Visiting: https://ieeexplore.ieee.org/xpl/tocresult.jsp?isnumber=22235&punumber=83
269 Visiting: https://ieeexplore.ieee.org/xpl/tocresult.jsp?isnumber=22336&punumber=83
270 Visiting: https://ieeexplore.ieee.org/xpl/tocresult.jsp?isnumber=24075&punumber=83
271 Visiting: https://ieeexplore.ieee.org/xpl/tocresult.jsp?isnumber=26424&punumber=83
272 Visiting: https://ieeexplore.ieee.org/xpl/tocresult.jsp?isnumber=15895&punumber=83
273 Visiting: https://ieeexplore.ieee.org/xpl/tocresult.jsp?isnumber=16029&punumber=83
274 Visiting: https://ieeexplore.ieee.org/xpl/tocresult.jsp?isnumber=16166&punumber=83
275 Visiting: https://ieeexplore.ieee.org/xpl/tocresult.jsp?isnumber=16278&punumber=83
276 Visiting: https://ieeexplore.ieee.org/x

359 Visiting: https://ieeexplore.ieee.org/xpl/tocresult.jsp?isnumber=7131605&punumber=83
360 Visiting: https://ieeexplore.ieee.org/xpl/tocresult.jsp?isnumber=7271151&punumber=83
361 Visiting: https://ieeexplore.ieee.org/xpl/tocresult.jsp?isnumber=6657713&punumber=83
362 Visiting: https://ieeexplore.ieee.org/xpl/tocresult.jsp?isnumber=6685907&punumber=83
363 Visiting: https://ieeexplore.ieee.org/xpl/tocresult.jsp?isnumber=6717077&punumber=83
364 Visiting: https://ieeexplore.ieee.org/xpl/tocresult.jsp?isnumber=6742656&punumber=83
365 Visiting: https://ieeexplore.ieee.org/xpl/tocresult.jsp?isnumber=6779706&punumber=83
366 Visiting: https://ieeexplore.ieee.org/xpl/tocresult.jsp?isnumber=6807541&punumber=83
367 Visiting: https://ieeexplore.ieee.org/xpl/tocresult.jsp?isnumber=6819102&punumber=83
368 Visiting: https://ieeexplore.ieee.org/xpl/tocresult.jsp?isnumber=6840896&punumber=83
369 Visiting: https://ieeexplore.ieee.org/xpl/tocresult.jsp?isnumber=6862127&punumber=83
370 Visiting: https:/

1031 Visiting: https://ieeexplore.ieee.org/xpl/tocresult.jsp?isnumber=4358840#pi-2013
1032 Visiting: https://ieeexplore.ieee.org/xpl/tocresult.jsp?isnumber=4358840#pi-2014
1033 Visiting: https://ieeexplore.ieee.org/xpl/tocresult.jsp?isnumber=4358840#pi-2015
1034 Visiting: https://ieeexplore.ieee.org/xpl/tocresult.jsp?isnumber=4358840#pi-2016
1035 Visiting: https://ieeexplore.ieee.org/xpl/tocresult.jsp?isnumber=4358840#pi-2017
1036 Visiting: https://ieeexplore.ieee.org/xpl/tocresult.jsp?isnumber=4358840#pi-2018
1037 Visiting: https://ieeexplore.ieee.org/xpl/tocresult.jsp?isnumber=4358840#pi-2019
1345 Visiting: https://ieeexplore.ieee.org/xpl/tocresult.jsp?isnumber=4358840#topic-1990s
1346 Visiting: https://ieeexplore.ieee.org/xpl/tocresult.jsp?isnumber=4358840#topic-2000s
1347 Visiting: https://ieeexplore.ieee.org/xpl/tocresult.jsp?isnumber=4358840#topic-2010s
1348 Visiting: https://ieeexplore.ieee.org/xpl/tocresult.jsp?isnumber=4358840#topic-1992
1349 Visiting: https://ieeexplore.ieee.

...

82607 Visiting: https://ieeexplore.ieee.org/document/4237201/
82608 Visiting: https://ieeexplore.ieee.org/document/4237201/citations#citations
82689 Visiting: https://ieeexplore.ieee.org/xpl/tocresult.jsp?isnumber=4271512&punumber=83#pi-1990s
82690 Visiting: https://ieeexplore.ieee.org/xpl/tocresult.jsp?isnumber=4271512&punumber=83#pi-2000s
82691 Visiting: https://ieeexplore.ieee.org/xpl/tocresult.jsp?isnumber=4271512&punumber=83#pi-2010s
82692 Visiting: https://ieeexplore.ieee.org/xpl/tocresult.jsp?isnumber=4271512&punumber=83#pi-1992
82693 Visiting: https://ieeexplore.ieee.org/xpl/tocresult.jsp?isnumber=4271512&punumber=83#pi-1993
82694 Visiting: https://ieeexplore.ieee.org/xpl/tocresult.jsp?isnumber=4271512&punumber=83#pi-1994
82695 Visiting: https://ieeexplore.ieee.org/xpl/tocresult.jsp?isnumber=4271512&punumber=83#pi-1995
82696 Visiting: https://ieeexplore.ieee.org/xpl/tocresult.jsp?isnumber=4271512&punumber=83#pi-1996
82697 Visiting: https://ieeexplore.ieee.org/xpl/tocresult.jsp?

83447 Visiting: https://ieeexplore.ieee.org/document/4271524/citations?tabFilter=patents
83453 Visiting: https://ieeexplore.ieee.org/document/4271525/
83454 Visiting: https://ieeexplore.ieee.org/document/4271525/citations#citations
83455 Visiting: https://ieeexplore.ieee.org/document/4271525/citations?tabFilter=patents
83461 Visiting: https://ieeexplore.ieee.org/document/4271515/
83462 Visiting: https://ieeexplore.ieee.org/document/4271515/citations#citations
83468 Visiting: https://ieeexplore.ieee.org/document/4271526/
83469 Visiting: https://ieeexplore.ieee.org/document/4271526/citations#citations
83483 Visiting: https://ieeexplore.ieee.org/document/4271530/
83484 Visiting: https://ieeexplore.ieee.org/document/4271530/citations#citations
83490 Visiting: https://ieeexplore.ieee.org/document/4271543/
83491 Visiting: https://ieeexplore.ieee.org/document/4271543/citations#citations
83497 Visiting: https://ieeexplore.ieee.org/document/4271529/
83498 Visiting: https://ieeexplore.ieee.org/d

84298 Visiting: https://ieeexplore.ieee.org/document/4287011/citations?tabFilter=patents
84304 Visiting: https://ieeexplore.ieee.org/document/4286997/
84305 Visiting: https://ieeexplore.ieee.org/document/4286997/citations#citations
84311 Visiting: https://ieeexplore.ieee.org/document/4287009/
84312 Visiting: https://ieeexplore.ieee.org/document/4287009/citations#citations
84313 Visiting: https://ieeexplore.ieee.org/document/4287009/citations?tabFilter=patents
84319 Visiting: https://ieeexplore.ieee.org/document/4286996/
84320 Visiting: https://ieeexplore.ieee.org/document/4286996/citations#citations
84326 Visiting: https://ieeexplore.ieee.org/document/4286994/
84327 Visiting: https://ieeexplore.ieee.org/document/4286994/citations#citations
84333 Visiting: https://ieeexplore.ieee.org/document/4286982/
84334 Visiting: https://ieeexplore.ieee.org/document/4286982/citations#citations
84340 Visiting: https://ieeexplore.ieee.org/document/4287010/
84341 Visiting: https://ieeexplore.ieee.org/d

84878 Visiting: https://ieeexplore.ieee.org/xpl/tocresult.jsp?isnumber=4303133&punumber=83#topic-2005
84879 Visiting: https://ieeexplore.ieee.org/xpl/tocresult.jsp?isnumber=4303133&punumber=83#topic-2006
84880 Visiting: https://ieeexplore.ieee.org/xpl/tocresult.jsp?isnumber=4303133&punumber=83#topic-2007
84881 Visiting: https://ieeexplore.ieee.org/xpl/tocresult.jsp?isnumber=4303133&punumber=83#topic-2008
84882 Visiting: https://ieeexplore.ieee.org/xpl/tocresult.jsp?isnumber=4303133&punumber=83#topic-2009
84883 Visiting: https://ieeexplore.ieee.org/xpl/tocresult.jsp?isnumber=4303133&punumber=83#topic-2010
84884 Visiting: https://ieeexplore.ieee.org/xpl/tocresult.jsp?isnumber=4303133&punumber=83#topic-2011
84885 Visiting: https://ieeexplore.ieee.org/xpl/tocresult.jsp?isnumber=4303133&punumber=83#topic-2012
84886 Visiting: https://ieeexplore.ieee.org/xpl/tocresult.jsp?isnumber=4303133&punumber=83#topic-2013
84887 Visiting: https://ieeexplore.ieee.org/xpl/tocresult.jsp?isnumber=4303133&pun

85757 Visiting: https://ieeexplore.ieee.org/xpl/tocresult.jsp?isnumber=4337753&punumber=83#topic-1992
85758 Visiting: https://ieeexplore.ieee.org/xpl/tocresult.jsp?isnumber=4337753&punumber=83#topic-1993
85759 Visiting: https://ieeexplore.ieee.org/xpl/tocresult.jsp?isnumber=4337753&punumber=83#topic-1994
85760 Visiting: https://ieeexplore.ieee.org/xpl/tocresult.jsp?isnumber=4337753&punumber=83#topic-1995
85761 Visiting: https://ieeexplore.ieee.org/xpl/tocresult.jsp?isnumber=4337753&punumber=83#topic-1996
85762 Visiting: https://ieeexplore.ieee.org/xpl/tocresult.jsp?isnumber=4337753&punumber=83#topic-1997
85763 Visiting: https://ieeexplore.ieee.org/xpl/tocresult.jsp?isnumber=4337753&punumber=83#topic-1998
85764 Visiting: https://ieeexplore.ieee.org/xpl/tocresult.jsp?isnumber=4337753&punumber=83#topic-1999
85765 Visiting: https://ieeexplore.ieee.org/xpl/tocresult.jsp?isnumber=4337753&punumber=83#topic-2000
85766 Visiting: https://ieeexplore.ieee.org/xpl/tocresult.jsp?isnumber=4337753&pun

In [ ]:
pagesVisited

In [ ]:
pagesToVisit

In [78]:
sm = 0
for i in pagesToVisit:
    for j in check:
        if j in i:
            sm += 1
#             print(i)
print(sm)

544


In [79]:
pagesToVisit

['https://ieeexplore.ieee.org/xpl/RecentIssue.jsp?punumber=83',
 'https://ieeexplore.ieee.org/xpl/RecentIssue.jsp?punumber=83',
 'http://www.ieee.org/',
 'http://standards.ieee.org/',
 'http://spectrum.ieee.org/',
 'http://www.ieee.org/sitemap.html',
 'https://www.ieee.org/cart/public/myCart/page.html?refSite=http://ieeexplore.ieee.org&refSiteName=IEEE%20Xplore',
 'https://ieeexplore.ieee.org/xpl/RecentIssue.jsp?punumber=83',
 'https://ieeexplore.ieee.org/xpl/RecentIssue.jsp?punumber=83',
 'https://www.ieee.org/security-privacy.html',
 'https://ieeexplore.ieee.org/xpl/RecentIssue.jsp?punumber=83',
 'https://ieeexplore.ieee.org/Xplore/home.jsp',
 "javascript:Modal.show('/xpl/mwInstSignIn.jsp')",
 'https://ieeexplore.ieee.org/browse/books/title/',
 'https://ieeexplore.ieee.org/browse/conferences/title/',
 'https://ieeexplore.ieee.org/xpl/courses.jsp',
 'https://ieeexplore.ieee.org/browse/periodicals/title/',
 'https://ieeexplore.ieee.org/browse/standards/collection/ieee',
 'https://ieeex